In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install datasets peft torch torchvision 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing

In [3]:
!pip install -q transformers torch evaluate scikit-learn pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.3 MB/s eta 0:00:00


In [4]:
import os
import sys
import subprocess

# 1️⃣ Uninstall any existing bitsandbytes build
#    to avoid mismatched CUDA binaries
print("🎯 Uninstalling old bitsandbytes...")
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "bitsandbytes"], check=True)

# 2️⃣ Install bitsandbytes >=0.43.2 which adds upstream support for CUDA 12.4+
#    (see HF docs: supports CUDA 11.0–12.5, including 12.4+) :contentReference[oaicite:0]{index=0}
print("🎯 Installing bitsandbytes>=0.43.2 for CUDA 12.4 support...")
subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "bitsandbytes>=0.43.2"], check=True)

# 3️⃣ Ensure the CUDA 12.4 runtime path is on LD_LIBRARY_PATH
#    so bitsandbytes can locate libcudart and its own libbitsandbytes_cuda124 :contentReference[oaicite:1]{index=1}
cuda_lib="/usr/local/cuda-12.4/lib64"
prev_ld = os.environ.get("LD_LIBRARY_PATH", "")
os.environ["LD_LIBRARY_PATH"] = f"{cuda_lib}:{prev_ld}"
print(f"🎯 LD_LIBRARY_PATH set to include {cuda_lib}")

# 4️⃣ Restart prompt reminder (mandatory in Colab to pick up the new .so files)
#    In standalone scripts this is not needed, but in notebooks you must restart runtime.
print("\n⚠️  If you’re in a Colab notebook, **please restart the runtime now**\n    (Runtime → Restart runtime) and re-run this cell/script.\n")

# 5️⃣ Diagnostic: confirm bitsandbytes is seeing CUDA 12.4
print("🔍 Running bitsandbytes diagnostic...")
res = subprocess.run([sys.executable, "-m", "bitsandbytes"], capture_output=True, text=True)
print(res.stdout)
if "CUDA" not in res.stdout:
    raise RuntimeError("bitsandbytes did not detect CUDA—check your LD_LIBRARY_PATH and installation.")


🎯 Uninstalling old bitsandbytes...


🎯 Installing bitsandbytes>=0.43.2 for CUDA 12.4 support...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 23.2 MB/s eta 0:00:00
🎯 LD_LIBRARY_PATH set to include /usr/local/cuda-12.4/lib64

⚠️  If you’re in a Colab notebook, **please restart the runtime now**
    (Runtime → Restart runtime) and re-run this cell/script.

🔍 Running bitsandbytes diagnostic...
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++ BUG REPORT INFORMATION ++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++ OTHER +++++++++++++++++++++++++++
CUDA specs: CUDASpecs(highest_compute_capability=(7, 5), cuda_version_string='124', cuda_version_tuple=(12, 4))
PyTorch settings found: CUDA_VERSION=124, Highest Compute Capability: (7, 5).
To manually override the PyTorch CUDA version please see: https://github.com/TimDettmers/bitsandbytes/blob/main/docs/source/nonpytorchcuda.mdx
CUDA SETUP: WARNING! CUDA runtime files not found in an

In [5]:
import os
import ast
import pandas as pd
from PIL import Image
import torch
from sklearn.model_selection import train_test_split  # scikit-learn split :contentReference[oaicite:2]{index=2}
import evaluate                                        # metrics :contentReference[oaicite:3]{index=3}
from transformers import AutoProcessor, AutoModelForVision2Seq , BitsAndBytesConfig, get_linear_schedule_with_warmup, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel, TaskType
from datasets import Dataset

CSV_PATH = "/kaggle/input/images-with-vqas/train_80_withoutyesno.csv"
train_df = pd.read_csv(CSV_PATH)

train_df["image_path"] = train_df["image_path"].str.replace(
    r"^Dataset/final_dataset/",
    "/kaggle/input/images-with-vqas/final_dataset/final_dataset/",
    regex=True
)

train_df["vqa_response"] = train_df["vqa_response"].apply(ast.literal_eval)
train_df = train_df.explode("vqa_response").reset_index(drop=True)
train_df[["question", "answer"]] = pd.DataFrame(train_df["vqa_response"].tolist(), index=train_df.index)

exists = train_df["image_path"].apply(os.path.exists)
print(f"Skipping {(~exists).sum()} rows with missing images")
train_df = train_df[exists]

train_df = train_df[["image_path", "question", "answer"]].reset_index(drop=True)
print(f"Total examples: {len(train_df)}")

2025-05-17 11:18:42.955478: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747480723.386032      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747480723.502268      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Skipping 0 rows with missing images
Total examples: 38906


In [6]:
CSV_PATH = "/kaggle/input/images-with-vqas/test_20_withoutyesno.csv"
test_df = pd.read_csv(CSV_PATH)

test_df["image_path"] = test_df["image_path"].str.replace(
    r"^Dataset/final_dataset/",
    "/kaggle/input/images-with-vqas/final_dataset/final_dataset/",
    regex=True
)

test_df["vqa_response"] = test_df["vqa_response"].apply(ast.literal_eval)
test_df = test_df.explode("vqa_response").reset_index(drop=True)
test_df[["question", "answer"]] = pd.DataFrame(test_df["vqa_response"].tolist(), index=test_df.index)

exists = test_df["image_path"].apply(os.path.exists)
print(f"Skipping {(~exists).sum()} rows with missing images")
test_df = test_df[exists]

test_df = test_df[["image_path", "question", "answer"]].reset_index(drop=True)
print(f"Total examples: {len(test_df)}")

Skipping 0 rows with missing images
Total examples: 9760


In [7]:
import os
import pandas as pd
from PIL import Image
from transformers import AutoProcessor, AutoModelForVisualQuestionAnswering, Trainer, TrainingArguments
import torch
from accelerate import Accelerator
from sklearn.model_selection import train_test_split
from peft import LoraConfig, get_peft_model
from transformers.data.data_collator import default_data_collator

# Initialize Accelerator for efficient multi-GPU training
accelerator = Accelerator()

# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_compute_dtype=torch.float16,
#     bnb_4bit_quant_type="nf4",
# )

# Load BLIP-1 VQA processor and model with fast image processing
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

base_model = AutoModelForVisualQuestionAnswering.from_pretrained(
    "Salesforce/blip-vqa-base",
    #quantization_config=quantization_config,
).to(device)

processor = AutoProcessor.from_pretrained("Salesforce/blip-vqa-base", use_fast=True)

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [8]:
model   = PeftModel.from_pretrained(
                   base_model, 
                   "/kaggle/input/blip-vqa-lora-combined/pytorch/default/1")  # same folder you saved earlier

model.to(device).eval()

PeftModel(
  (base_model): LoraModel(
    (model): BlipForQuestionAnswering(
      (vision_model): BlipVisionModel(
        (embeddings): BlipVisionEmbeddings(
          (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
        )
        (encoder): BlipEncoder(
          (layers): ModuleList(
            (0-11): 12 x BlipEncoderLayer(
              (self_attn): BlipAttention(
                (dropout): Dropout(p=0.0, inplace=False)
                (qkv): Linear(in_features=768, out_features=2304, bias=True)
                (projection): Linear(in_features=768, out_features=768, bias=True)
              )
              (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
              (mlp): BlipMLP(
                (activation_fn): GELUActivation()
                (fc1): lora.Linear(
                  (base_layer): Linear(in_features=768, out_features=3072, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dro

In [9]:
from tqdm import tqdm
preds, refs = [], []
for row in tqdm(test_df.itertuples(), total = len(test_df)):
    img = Image.open(row.image_path).convert("RGB")
    prompt = f"Based on the image, answer the following question with a single word. Question: {row.question} Answer:"
    inputs = processor(images=img, text=prompt, return_tensors="pt").to(device)
    pixel_values   = inputs["pixel_values"]
    input_ids      = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]
    out = model.generate(
        pixel_values=pixel_values,
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=10
    )
    text = processor.batch_decode(out, skip_special_tokens=True)[0].strip()
    pred = text.split()[0].rstrip(".,;:!?") if text else ""
    preds.append(pred.lower())
    refs.append(row.answer.lower())

100%|██████████| 9760/9760 [16:15<00:00, 10.00it/s]


In [11]:
# ──────────────────────────────────────────────────────────────────────────────
# 6. Compute Metrics (and identify mismatches)
# ──────────────────────────────────────────────────────────────────────────────
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import evaluate
# from bert_score import BERTScorer # Keep if you add BERTScore later
# from rouge_score import rouge_scorer # Keep if you add ROUGE later
# from rapidfuzz.distance import Levenshtein # Keep if you add Levenshtein later
from sentence_transformers import SentenceTransformer, util # Keep if you add SBERT later
import numpy as np
import pandas as pd # Ensure pandas is imported if df is used



# normalize case
preds_l = [str(p).lower().strip() for p in preds] # Added str() and strip() for robustness
refs_l  = [str(r).lower().strip() for r in refs]   # Added str() and strip()

# # 1) TOKEN-LEVEL EXACT MATCH → binary labels
y_pred_bin = [int(p == r) for p, r in zip(preds_l, refs_l)]
y_true_bin = [1]*len(refs_l)            # reference is always “correct” class

acc = accuracy_score(y_true_bin, y_pred_bin)
prec, rec, f1, _ = precision_recall_fscore_support(
    y_true_bin, y_pred_bin, average="binary", zero_division=0
)
print(f"Exact-match Accuracy: {acc:.3f}")
print(f"Exact-match Precision: {prec:.3f}") # Will be 1.0 if acc > 0, else 0.0
print(f"Exact-match Recall:    {rec:.3f}")    # Will be same as acc
print(f"Exact-match F1:        {f1:.3f}\n")  

Exact-match Accuracy: 0.570
Exact-match Precision: 1.000
Exact-match Recall:    0.570
Exact-match F1:        0.726



In [12]:
! pip install git+https://github.com/google-research/bleurt.git
!pip install scikit-learn evaluate bert-score rouge-score \
            sentence-transformers rapidfuzz

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-56wad70r
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt.git /tmp/pip-req-build-56wad70r
  Resolved https://github.com/google-research/bleurt.git to commit cebe7e6f996b40910cfaa520a63db47807e3bf5c
  Preparing metadata (setup.py) ... done
  Created wheel for BLEURT: filename=BLEURT-0.0.2-py3-none-any.whl size=16456766 sha256=eb1dd62aaab34c010d2ef5167091125e7d3b795b7ad2d48c7eeaaff895b6eeef
  Stored in directory: /tmp/pip-ephem-wheel-cache-5n6i6cey/wheels/30/af/34/e148007788b060e4c76e7ecf68e70c692dff0f2632e62ac454
Successfully built BLEURT


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 35.8 MB/s eta 0:00:0000:0100:01
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=971bbf96983472cfc6ece400561309472ebbd9d63e514587e3c2709a691cce96
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [13]:
from bert_score import BERTScorer # Keep if you add BERTScore later
from rouge_score import rouge_scorer # Keep if you add ROUGE later
from rapidfuzz.distance import Levenshtein # Keep if you add Levenshtein later
from sentence_transformers import SentenceTransformer, util # Keep if you add SBERT later

# --- ROUGE ---
print("\n--- ROUGE Scores ---")
rouge_eval_scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
rouge1_scores = []
rougeL_scores = []
for pred, ref in zip(preds_l, refs_l):
    if not pred or not ref: # Handle empty strings if any
        rouge1_scores.append(0.0)
        rougeL_scores.append(0.0)
        continue
    scores = rouge_eval_scorer.score(ref, pred) # Target, Prediction
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)

if rouge1_scores:
    print(f"Average ROUGE-1 F1: {np.mean(rouge1_scores):.3f}")
    print(f"Average ROUGE-L F1: {np.mean(rougeL_scores):.3f}")
else:
    print("No ROUGE scores to compute (empty predictions or references).")


# --- BERTScore ---
print("\n--- BERTScore ---")
try:
    bert_eval_scorer = BERTScorer(lang="en", rescale_with_baseline=True, device=device)
    filtered_preds_l = [p for p, r in zip(preds_l, refs_l) if p and r]
    filtered_refs_l = [r for p, r in zip(preds_l, refs_l) if p and r]

    if filtered_preds_l and filtered_refs_l:
        P, R, F1_bert = bert_eval_scorer.score(filtered_preds_l, filtered_refs_l)
        print(f"Average BERTScore Precision: {P.mean():.3f}")
        print(f"Average BERTScore Recall:    {R.mean():.3f}")
        print(f"Average BERTScore F1:        {F1_bert.mean():.3f}")
    else:
        print("Not enough valid (non-empty) prediction/reference pairs for BERTScore.")
except Exception as e:
    print(f"Could not compute BERTScore: {e}")


# --- Levenshtein Normalized Similarity ---
print("\n--- Levenshtein Normalized Similarity ---")
lev_similarities = []
for pred, ref in zip(preds_l, refs_l):
    if not pred and not ref: # both empty, perfect match
        similarity = 1.0
    elif not pred or not ref: # one empty, other not, zero similarity
        similarity = 0.0
    else:
        similarity = Levenshtein.normalized_similarity(pred, ref)
    lev_similarities.append(similarity)

if lev_similarities:
    print(f"Average Levenshtein Normalized Similarity: {np.mean(lev_similarities):.3f}")
else:
    print("No Levenshtein similarities to compute.")


# --- Sentence Transformer Cosine Similarity ---
print("\n--- Sentence-BERT Cosine Similarity ---")
try:
    sbert_model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
    valid_pairs_indices = [i for i, (p, r) in enumerate(zip(preds_l, refs_l)) if p and r]
    sbert_preds = [preds_l[i] for i in valid_pairs_indices]
    sbert_refs = [refs_l[i] for i in valid_pairs_indices]

    if sbert_preds and sbert_refs:
        embeddings_preds = sbert_model.encode(sbert_preds, convert_to_tensor=True)
        embeddings_refs = sbert_model.encode(sbert_refs, convert_to_tensor=True)

        cosine_scores_sbert = util.cos_sim(embeddings_preds, embeddings_refs)
        pair_similarities = [cosine_scores_sbert[i, i].item() for i in range(len(sbert_preds))]
        print(f"Average Sentence-BERT Cosine Similarity: {np.mean(pair_similarities):.3f}")
    else:
        print("Not enough valid (non-empty) prediction/reference pairs for Sentence-BERT similarity.")
except Exception as e:
    print(f"Could not compute Sentence-BERT similarity: {e}")


--- ROUGE Scores ---
Average ROUGE-1 F1: 0.580
Average ROUGE-L F1: 0.580

--- BERTScore ---


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/305 [00:00<?, ?it/s]

Batches:   0%|          | 0/305 [00:00<?, ?it/s]

Average Sentence-BERT Cosine Similarity: 0.801
